In [2]:
import os
import pandas as pd
os.chdir('/Users/amitagarwal/GreyAtom/Hackathon/Hotstar/')

This is a multi-stage problem. Essentially we should be building a classifier that would classify the text cases' sentiment as pos or neg. The pdf outlines that we are supposed to cluster the customers, which we would do.

In [7]:
#import train data and clean the data first.
df = pd.read_json("/Users/amitagarwal/GreyAtom/Hackathon/Hotstar/data/train_data.json")
train_copy = df.copy()

In [8]:
df.head(4)

,train-121672,train-121673,train-121670,train-121671,train-121676,train-121677,train-121674,train-121675,train-121678,train-121679,...,train-126321,train-126320,train-126323,train-126322,train-126325,train-126324,train-126327,train-126326,train-126329,train-126328
cities,"delhi:3492,mumbai:64","ahmedabad:56666,mumbai:26989",navi mumbai:29400,"chennai:77,navi mumbai:4005,mumbai:0","bangalore:5135,krishnarajapura:529,hosur:2864",bangalore:5,"gurgaon:12635,delhi:13142,nagari:3885","bangalore:373,chennai:6205,mumbai:77","gurgaon:1124,delhi:9492,mumbai:19504","rawalpindi:976,lahore:104",...,bangalore:1120,gurgaon:31707,mumbai:31082,"delhi:1620,navi mumbai:8848",mumbai:14760,"khanna:2295,mumbai:1548","lisboa:3756,porto:3198",kolkata:25436,"chennai:7999,delhi:8945,ahmedabad:33811",navi mumbai:49286
dow,"5:482,4:3008,7:64","1:3257,3:9162,2:12893,5:9237,4:8503,7:25094,6:...","1:40,3:2700,2:4659,5:7338,4:6641,7:2218,6:5800","1:77,2:3883,5:122","1:6150,3:2378",2:5,"1:15909,3:2465,2:5580,4:293,7:1283,6:4131","3:5481,4:724,7:373,6:77","1:6965,3:21,2:6183,5:1434,4:6995,7:4599,6:3923","1:1043,7:37",...,"5:50,7:1069","1:6095,3:5684,2:2943,5:6635,4:2750,7:2274,6:5316","1:6255,3:10275,2:6019,4:197,7:6901,6:1435","1:229,5:2196,7:8022,6:20","1:1333,3:147,2:249,5:304,4:7825,7:4900","1:1108,5:40,4:220,7:2472","1:948,5:2317,7:3688","1:2562,3:3153,2:5009,5:3875,4:4274,7:3284,6:3278","1:11880,3:7673,2:17472,5:2759,4:5122,7:4836,6:...","1:11425,3:7989,2:6421,5:4460,4:4157,7:7896,6:6928"
genres,"Drama:6,Cricket:3469,Wildlife:79","Drama:48949,Family:9927,Crime:1901,Romance:165...","Cricket:5715,Family:2777,Drama:20873,LiveTV:4,...","Romance:77,Drama:122,Cricket:3883",TalkShow:8529,TalkShow:5,"Cricket:0,Wildlife:21,Crime:29591,Drama:10,Liv...","Romance:724,Cricket:5854,Reality:77","Drama:10090,Cricket:1124,Comedy:18907",Family:1080,...,"Drama:50,Action:582,Comedy:487","Cricket:13504,Family:17267,Reality:151,Romance...",Drama:31082,"Cricket:450,Kids:6,TalkShow:20,Wildlife:40,Rom...","Cricket:14345,Badminton:97,Reality:69,Crime:249","Cricket:3787,Reality:57","Cricket:5060,Awards:21,Crime:1872","Drama:25428,Family:7","Drama:50749,Romance:9","Cricket:17417,Reality:14947,Drama:14565,LiveTV..."
segment,neg,neg,neg,neg,pos,neg,neg,neg,neg,neg,...,neg,neg,neg,neg,neg,neg,neg,neg,neg,neg


In [9]:
#Transposing this
df = df.T

In [10]:
#select first 500 rows.
df_500 = df.iloc[:500]

In [11]:
df_500.shape

(500, 6)

Start off with the cities column. Find the total number of cities that are present, and then do a 1-hot encoding for these, giving the seconds value for the cities which have seconds value for particluar users and 0 for the rest. This should make sense intuitively. 

In [13]:
df_500["cities"]

train-121672                                 delhi:3492,mumbai:64
train-121673                         ahmedabad:56666,mumbai:26989
train-121670                                    navi mumbai:29400
train-121671                 chennai:77,navi mumbai:4005,mumbai:0
train-121676        bangalore:5135,krishnarajapura:529,hosur:2864
train-121677                                          bangalore:5
train-121674                gurgaon:12635,delhi:13142,nagari:3885
train-121675                 bangalore:373,chennai:6205,mumbai:77
train-121678                 gurgaon:1124,delhi:9492,mumbai:19504
train-121679                            rawalpindi:976,lahore:104
train-199734                  kolkata:16889,delhi:180,chennai:245
train-193747    kolkata:143,gurgaon:84201,chennai:25671,mumbai...
train-123292                              kolkata:2490,mumbai:329
train-123293                                           mumbai:990
train-123290                            delhi:4655,navi mumbai:22
train-1232

Before that the initial dataframe needs some cleaning. 

We will remove the ridiculuos index columns, add a new column called user

In [15]:
import warnings
warnings.filterwarnings("ignore")
df_500["index"] = df_500.index

In [16]:
df_500["user"] = df_500["index"].str.replace("train-","")

We will encode segment column to pos = 1, and neg = 0

In [17]:
df_500["segment"] = df_500["segment"].str.replace("neg","0")
df_500["segment"] = df_500["segment"].str.replace("pos","1")

In [19]:
df_500["segment"].value_counts() #Imbalanced Dataset

0    442
1     58
Name: segment, dtype: int64

Now, for the first user that is 121672, the idea is to segregate each of his cities, his time of days, his days of week, his genres into seconds

Something which should look like this finally for user 1. The column names are the various cities and times, and days possible for all the users in the dataframe and the values are 0s if no value exists for that particular column for that user and a numeric if that city or time has a second value associated with it for that user. Similar to One hot encoding. or rather the pd.get_dummies.

In [24]:
cities = df_500["cities"].values.tolist()

In [25]:
cities

['delhi:3492,mumbai:64',
 'ahmedabad:56666,mumbai:26989',
 'navi mumbai:29400',
 'chennai:77,navi mumbai:4005,mumbai:0',
 'bangalore:5135,krishnarajapura:529,hosur:2864',
 'bangalore:5',
 'gurgaon:12635,delhi:13142,nagari:3885',
 'bangalore:373,chennai:6205,mumbai:77',
 'gurgaon:1124,delhi:9492,mumbai:19504',
 'rawalpindi:976,lahore:104',
 'kolkata:16889,delhi:180,chennai:245',
 'kolkata:143,gurgaon:84201,chennai:25671,mumbai:72465',
 'kolkata:2490,mumbai:329',
 'mumbai:990',
 'delhi:4655,navi mumbai:22',
 'pune:33,chennai:38097,navi mumbai:28054,mumbai:2217',
 'navi mumbai:3810',
 'pune:17288,delhi:5139,gurgaon:12,mumbai:2111',
 'mumbai:37',
 'new delhi:58,delhi:4162,ahmedabad:23408,mumbai:18809',
 'hyderabad:1026',
 'delhi:1286,navi mumbai:202723,mumbai:1508',
 'navi mumbai:10667',
 'chennai:7895,delhi:258,nagar:375',
 'ludhiana:4543',
 'kolkata:984,bangalore:55260',
 'new delhi:6418,gurgaon:74,delhi:3715,bangalore:776,mumbai:5608',
 'bangalore:21889,chennai:2505',
 'mumbai:28855',
 

In [26]:
first_row = cities[0].split(",")

In [27]:
first_row

['delhi:3492', 'mumbai:64']

In [32]:
flat_list_1 = [item for sublist in [i.split(':', 1) for i in first_row] for item in sublist]

In [33]:
flat_list_1

['delhi', '3492', 'mumbai', '64']

In [34]:
city = pd.DataFrame(flat_list_1[0::2]).T
second = pd.DataFrame(flat_list_1[1::2]).T

In [35]:
city

,0,1
0,delhi,mumbai


In [36]:
second

,0,1
0,3492,64


In [37]:
s = pd.concat([city,second],ignore_index=True) #ignoring index is key, you will need them to reset to indexes 0 and 1
#will save countless minutes of pain, trying to figre out where the nans during concatenation came from. indexes are 
#key during a concat. if the dfs do not have similar indexes they will not concat accordingly 



In [38]:
s

,0,1
0,delhi,mumbai
1,3492,64


In [39]:
s.columns = s.iloc[0] #making the first row or .iloc[0] as the header
s = s.drop([0],0) #dropping .iloc[0]th row

In [40]:
s #This is exactly what we wanted. Now all we have to do is to write a function which should do the same 
#for all the rows in the cities column for every user. 

,delhi,mumbai
1,3492,64


In [41]:
#Doing something similar for the second row before we write the function, should make this crystal. 

In [42]:
second_row = cities[1].split(",")

In [43]:
second_row

['ahmedabad:56666', 'mumbai:26989']

In [44]:
flat_list_2 = [item for sublist in [i.split(':', 1) for i in second_row] for item in sublist]

In [45]:
flat_list_2

['ahmedabad', '56666', 'mumbai', '26989']

In [46]:
city = pd.DataFrame(flat_list_2[0::2]).T
second = pd.DataFrame(flat_list_2[1::2]).T
f = pd.concat([city,second],ignore_index=True) 

In [47]:
f

,0,1
0,ahmedabad,mumbai
1,56666,26989


In [48]:
f.columns = f.iloc[0] #making the first row or .iloc[0] as the header
f = f.drop([0],0) #dropping .iloc[0]th row

In [49]:
s

,delhi,mumbai
1,3492,64


In [50]:
frames = [s,f]
fs = pd.concat(frames,join='outer', ignore_index=True, sort=False)

In [51]:
fs

,delhi,mumbai,ahmedabad
0,3492,64,NaN
1,NaN,26989,56666


In [53]:
fs.fillna(0)

,delhi,mumbai,ahmedabad
0,3492,64,0
1,0,26989,56666


In [54]:
#This is exactly what we are after. The above represents just how the user 1 and 2 should have columns 
#instead of the cities

#Getting this to a for loop

In [55]:
final=[]
for i in range(len(cities)):
    frames=[]
    user_cities = cities[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_cities] for item in sublist]
    city = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([city,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True, sort=False)
    final.append(pd.DataFrame(final_df))
    
city = pd.concat(final,join='outer', ignore_index=True, sort=False)
city = city.fillna(0) 

In [56]:
city.head(10) #Nice!

,delhi,mumbai,ahmedabad,navi mumbai,chennai,bangalore,krishnarajapura,hosur,gurgaon,nagari,...,surat,thane,vasai,perth,singapore,ranchi,madrid,barcelona,bhubaneswar,al kuwayt
0,3492,64,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,26989,56666,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,29400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,4005,77,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,5135,529,2864,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,13142,0,0,0,0,0,0,0,12635,3885,...,0,0,0,0,0,0,0,0,0,0
7,0,77,0,0,6205,373,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9492,19504,0,0,0,0,0,0,1124,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Doing something exactly similar for genres

In [57]:
genres = df_500["genres"].values.tolist()

In [58]:
genres

['Drama:6,Cricket:3469,Wildlife:79',
 'Drama:48949,Family:9927,Crime:1901,Romance:16571,Action:2064,Comedy:4225',
 'Cricket:5715,Family:2777,Drama:20873,LiveTV:4,Action:14,Comedy:15',
 'Romance:77,Drama:122,Cricket:3883',
 'TalkShow:8529',
 'TalkShow:5',
 'Cricket:0,Wildlife:21,Crime:29591,Drama:10,LiveTV:4,Comedy:35',
 'Romance:724,Cricket:5854,Reality:77',
 'Drama:10090,Cricket:1124,Comedy:18907',
 'Family:1080',
 'Romance:1889,Cricket:739,TalkShow:605,Wildlife:52,Crime:3524,Drama:9725,Action:172,Comedy:602',
 'Drama:57268,Cricket:30,Family:70740,Romance:54428,Action:5,Thriller:4',
 'Cricket:2745,Reality:74',
 'TalkShow:1,Family:818,Reality:171',
 'Romance:116,Cricket:4533,Football:24,Badminton:2',
 'Cricket:61889,Sport:1,TalkShow:1758,Crime:19,Romance:29,LiveTV:3913,Reality:790',
 'Action:3810',
 'Drama:6551,Cricket:15808,Football:160,Thriller:3,Reality:2024',
 'Action:37',
 'Cricket:1794,TalkShow:710,Football:36,Reality:17053,Drama:26610,Awards:232',
 'Romance:1026',
 'Drama:13815,

In [59]:
final_1=[]
for i in range(len(genres)):
    frames_genres=[]
    genre = genres[i].split(",")
    flat_list_g = [item for sublist in [i.split(':', 1) for i in genre] for item in sublist]
    flat_list_g = [i.lower() for i in flat_list_g]
    genre_1 = pd.DataFrame(flat_list_g[0::2]).T
    second_g = pd.DataFrame(flat_list_g[1::2]).T
    genre_df = pd.concat([genre_1,second_g],ignore_index = True)
    genre_df.columns = genre_df.iloc[0]
    genre_df = genre_df.drop([0],0)
    frames_genres.append(genre_df)
    final_df_g = pd.concat(frames_genres,join='outer', ignore_index=True, sort=False)
    final_1.append(pd.DataFrame(final_df_g))

In [60]:
genre = pd.concat(final_1,join='outer', ignore_index=True, sort=False)
genre = genre.fillna(0) 

In [61]:
genre.head(10)

,drama,cricket,wildlife,family,crime,romance,action,comedy,livetv,talkshow,...,travel,teen,mythology,kids,formula1,science,horror,hockey,kabaddi,table tennis
0,6,3469,79,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,48949,0,0,9927,1901,16571,2064,4225,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20873,5715,0,2777,0,0,14,15,4,0,...,0,0,0,0,0,0,0,0,0,0
3,122,3883,0,0,0,77,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,8529,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
6,10,0,21,0,29591,0,0,35,4,0,...,0,0,0,0,0,0,0,0,0,0
7,0,5854,0,0,0,724,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,10090,1124,0,0,0,0,0,18907,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1080,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
#We will have to do the same for tod and dow, however there is a catch involved. 

#Starting off with the dow, the days are labelled as 1,2,3,4,..... We will have to rename them. 

#1 as Sunday, 2 as Monday and so. So we will have to rename the columns from a dictionary

In [63]:
days_of_week = df_500["dow"].values.tolist()

In [64]:
days_of_week

['5:482,4:3008,7:64',
 '1:3257,3:9162,2:12893,5:9237,4:8503,7:25094,6:15493',
 '1:40,3:2700,2:4659,5:7338,4:6641,7:2218,6:5800',
 '1:77,2:3883,5:122',
 '1:6150,3:2378',
 '2:5',
 '1:15909,3:2465,2:5580,4:293,7:1283,6:4131',
 '3:5481,4:724,7:373,6:77',
 '1:6965,3:21,2:6183,5:1434,4:6995,7:4599,6:3923',
 '1:1043,7:37',
 '1:275,3:1090,2:4390,5:25,4:2796,6:8732',
 '1:15578,3:19502,2:21455,5:31126,4:29995,7:23403,6:41414',
 '4:1067,7:1752',
 '1:188,2:801',
 '5:20,4:2052,7:2603',
 '1:6845,3:610,2:7529,5:14750,4:14103,7:8369,6:16189',
 '5:3810',
 '1:311,2:980,5:1136,4:6330,7:9249,6:6538',
 '1:37',
 '1:15604,3:3852,2:15275,5:2613,4:2805,7:4681,6:1605',
 '1:1026',
 '1:28375,3:35336,2:20357,5:28410,4:24327,7:40775,6:27927',
 '3:7447,2:86,5:2777,4:70,6:287',
 '1:387,3:1792,2:258,4:6091',
 '1:645,5:3896',
 '3:2107,2:1341,5:6904,4:168,7:32469,6:13252',
 '1:2051,3:4758,2:1832,5:1166,4:2974,7:1590,6:2217',
 '1:822,2:5761,5:953,4:3376,7:5804,6:7675',
 '1:2706,3:4753,2:4855,5:5582,4:2356,7:8599',
 '1:32

In [65]:
final_2=[]
for i in range(len(days_of_week)):
    frames=[]
    user_days = days_of_week[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_days] for item in sublist]
#    flat_list = [int(i) for i in flat_list]
    day = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([day,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True, sort=False)
    final_2.append(pd.DataFrame(final_df))

days = pd.concat(final_2,join='outer', ignore_index=True, sort=False)
days = days.fillna(0)

In [66]:
days.head()

,5,4,7,1,3,2,6
0,482,3008,64,0,0,0,0
1,9237,8503,25094,3257,9162,12893,15493
2,7338,6641,2218,40,2700,4659,5800
3,122,0,0,77,0,3883,0
4,0,0,0,6150,2378,0,0


In [67]:
days_dict = {"1":"Sunday", "2":"Monday", "3":"Tuesday", "4":"Wednesday", "5":"Thursday", 
             "6":"Friday", "7":"Saturday"} 

In [68]:
days_dict

{'1': 'Sunday',
 '2': 'Monday',
 '3': 'Tuesday',
 '4': 'Wednesday',
 '5': 'Thursday',
 '6': 'Friday',
 '7': 'Saturday'}

In [69]:
day_key = pd.DataFrame(list(days_dict.items()),columns = ["dow_cat","day"])  
d = day_key.set_index('dow_cat').to_dict()  
days.columns = days.columns.to_series().map(d['day'])

In [70]:
days.head(10)

,Thursday,Wednesday,Saturday,Sunday,Tuesday,Monday,Friday
0,482,3008,64,0,0,0,0
1,9237,8503,25094,3257,9162,12893,15493
2,7338,6641,2218,40,2700,4659,5800
3,122,0,0,77,0,3883,0
4,0,0,0,6150,2378,0,0
5,0,0,0,0,0,5,0
6,0,293,1283,15909,2465,5580,4131
7,0,724,373,0,5481,0,77
8,1434,6995,4599,6965,21,6183,3923
9,0,0,37,1043,0,0,0


In [71]:
#Will have to repeat this for tod. Now TOD has values ranging from 0 to 23 for the 24 hours of the day

# We will have to use a dictionary again

#time_dict = {"0":"12 am",
#             "1":"1 am"
#             "2":"2 am"
#             "3":"3 am"
#             "4":"4 am"
#        ....}

#Making one manually is painful, so we will use list comprehensions

In [72]:
time_am = [str(x) for x in range(0,12)]
time_pm=[str(x) for x in range(12,24)]
am=[str(x)+ " am" for x in range(0,12)]
pm=[str(x)+ " pm" for x in range(0,12)]
d1 = dict(zip(time_am,am))
d2 = dict(zip(time_pm,pm))
d1["0"] = "12 am" #changing 0 to 12 am
d2["12"] = "12 pm" #changing 12 to 12 pm 
time_dict = {**d1,**d2} #merging both dicts

In [73]:
time_dict

{'0': '12 am',
 '1': '1 am',
 '2': '2 am',
 '3': '3 am',
 '4': '4 am',
 '5': '5 am',
 '6': '6 am',
 '7': '7 am',
 '8': '8 am',
 '9': '9 am',
 '10': '10 am',
 '11': '11 am',
 '12': '12 pm',
 '13': '1 pm',
 '14': '2 pm',
 '15': '3 pm',
 '16': '4 pm',
 '17': '5 pm',
 '18': '6 pm',
 '19': '7 pm',
 '20': '8 pm',
 '21': '9 pm',
 '22': '10 pm',
 '23': '11 pm'}

In [74]:
time_of_day = df_500["tod"].values.tolist()

In [75]:
time_of_day

['18:1372,21:1862,17:320',
 '11:7726,10:3521,13:2067,12:3844,20:4608,21:9360,17:5734,16:4438,19:3822,18:4274,23:2988,22:304,1:7419,0:7944,2:9807,9:2005,15:967,14:2807',
 '11:302,10:1594,13:331,12:133,20:3802,14:3083,22:2219,23:2111,19:3683,18:101,8:201,7:504,6:1874,9:4388,15:1640,21:3430',
 '10:122,12:77,15:1800,14:0,16:2083',
 '20:529,21:1257,22:4930,23:538,19:1268,18:6',
 '10:5',
 '10:21,13:4576,12:3284,15:2158,14:4446,22:2839,23:7937,1:10,0:1252,5:1490,6:1648,9:0',
 '19:4245,20:2410',
 '10:224,20:691,14:683,17:1031,16:3,1:15312,0:5051,3:2556,2:4559,8:10',
 '19:976,13:67,22:37',
 '11:281,10:775,13:735,15:213,21:225,22:2907,23:632,19:182,1:4739,0:4150,2:810,17:7,8:0,9:1228,20:159,14:262',
 '11:3152,13:14973,12:3234,15:13363,14:34461,22:4358,23:11031,16:151,1:42489,0:45814,2:5553,20:793,21:3102',
 '18:329,20:1067,21:1349,22:74',
 '0:188,3:1,15:800',
 '15:131,14:15,17:948,16:978,19:2,18:1277,22:12,20:368,21:944',
 '11:1976,10:7390,13:3332,12:145,15:5329,14:3853,17:4458,16:4236,19:5598,1

In [76]:
final_3=[]
for i in range(len(days_of_week)):
    frames=[]
    user_time = time_of_day[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_time] for item in sublist]
#    flat_list = [int(i) for i in flat_list]
    time = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([time,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True, sort=False)
    final_3.append(pd.DataFrame(final_df))
timeofday = pd.concat(final_3,join='outer', ignore_index=True, sort=False)
timeofday = timeofday.fillna(0)
time_key = pd.DataFrame(list(time_dict.items()),columns = ["tod_cat","hour"]) 
t = time_key.set_index('tod_cat').to_dict() 
timeofday.columns = timeofday.columns.to_series().map(t['hour']) 
time_of_day = timeofday

In [77]:
time_of_day.head(10) #Great

,6 pm,9 pm,5 pm,11 am,10 am,1 pm,12 pm,8 pm,4 pm,7 pm,...,2 am,9 am,3 pm,2 pm,8 am,7 am,6 am,5 am,3 am,4 am
0,1372,1862,320,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4274,9360,5734,7726,3521,2067,3844,4608,4438,3822,...,9807,2005,967,2807,0,0,0,0,0,0
2,101,3430,0,302,1594,331,133,3802,0,3683,...,0,4388,1640,3083,201,504,1874,0,0,0
3,0,0,0,0,122,0,77,0,2083,0,...,0,0,1800,0,0,0,0,0,0,0
4,6,1257,0,0,0,0,0,529,0,1268,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,21,4576,3284,0,0,0,...,0,0,2158,4446,0,0,1648,1490,0,0
7,0,0,0,0,0,0,0,2410,0,4245,...,0,0,0,0,0,0,0,0,0,0
8,0,0,1031,0,224,0,0,691,3,0,...,4559,0,0,683,10,0,0,0,2556,0
9,0,0,0,0,0,67,0,0,0,976,...,0,0,0,0,0,0,0,0,0,0


Now time to concat all these dataframes together: city, genres, days, and time_of_day with the original df and drop cities, dow, tod and genres from that as they are being replaced with these new dataframes which we just created. For the record we will also drop titles watched. Reason being recommendations (or clustering) should do fine for now with the genres. Titles might be a bit of an overkill at this point, we will try and keep this as generic as possible. A genre drill down into titles might not be necessary at this point. 

In [78]:
df_500.head()

#resetting index

,cities,dow,genres,segment,titles,tod,index,user
train-121672,"delhi:3492,mumbai:64","5:482,4:3008,7:64","Drama:6,Cricket:3469,Wildlife:79",0,"Wild Sex:79,Vintage Dhoni Comes Good with 134:...","18:1372,21:1862,17:320",train-121672,121672
train-121673,"ahmedabad:56666,mumbai:26989","1:3257,3:9162,2:12893,5:9237,4:8503,7:25094,6:...","Drama:48949,Family:9927,Crime:1901,Romance:165...",0,"Naamkarann:369,Ishq:59,Pardes Mein Hai Mera Di...","11:7726,10:3521,13:2067,12:3844,20:4608,21:936...",train-121673,121673
train-121670,navi mumbai:29400,"1:40,3:2700,2:4659,5:7338,4:6641,7:2218,6:5800","Cricket:5715,Family:2777,Drama:20873,LiveTV:4,...",0,"India vs Australia 1st Test Hindi:83,India vs ...","11:302,10:1594,13:331,12:133,20:3802,14:3083,2...",train-121670,121670
train-121671,"chennai:77,navi mumbai:4005,mumbai:0","1:77,2:3883,5:122","Romance:77,Drama:122,Cricket:3883",0,"Rangoon:77,India A vs England XI:3883,Kaabil:122","10:122,12:77,15:1800,14:0,16:2083",train-121671,121671
train-121676,"bangalore:5135,krishnarajapura:529,hosur:2864","1:6150,3:2378",TalkShow:8529,1,Koffee With Karan:8529,"20:529,21:1257,22:4930,23:538,19:1268,18:6",train-121676,121676


In [79]:
df_500 = df_500.reset_index()

In [80]:
df_500.head(4)

,level_0,cities,dow,genres,segment,titles,tod,index,user
0,train-121672,"delhi:3492,mumbai:64","5:482,4:3008,7:64","Drama:6,Cricket:3469,Wildlife:79",0,"Wild Sex:79,Vintage Dhoni Comes Good with 134:...","18:1372,21:1862,17:320",train-121672,121672
1,train-121673,"ahmedabad:56666,mumbai:26989","1:3257,3:9162,2:12893,5:9237,4:8503,7:25094,6:...","Drama:48949,Family:9927,Crime:1901,Romance:165...",0,"Naamkarann:369,Ishq:59,Pardes Mein Hai Mera Di...","11:7726,10:3521,13:2067,12:3844,20:4608,21:936...",train-121673,121673
2,train-121670,navi mumbai:29400,"1:40,3:2700,2:4659,5:7338,4:6641,7:2218,6:5800","Cricket:5715,Family:2777,Drama:20873,LiveTV:4,...",0,"India vs Australia 1st Test Hindi:83,India vs ...","11:302,10:1594,13:331,12:133,20:3802,14:3083,2...",train-121670,121670
3,train-121671,"chennai:77,navi mumbai:4005,mumbai:0","1:77,2:3883,5:122","Romance:77,Drama:122,Cricket:3883",0,"Rangoon:77,India A vs England XI:3883,Kaabil:122","10:122,12:77,15:1800,14:0,16:2083",train-121671,121671


In [81]:
df_500 = df_500.drop(["level_0","index"],1)

In [82]:
df_500.head()

,cities,dow,genres,segment,titles,tod,user
0,"delhi:3492,mumbai:64","5:482,4:3008,7:64","Drama:6,Cricket:3469,Wildlife:79",0,"Wild Sex:79,Vintage Dhoni Comes Good with 134:...","18:1372,21:1862,17:320",121672
1,"ahmedabad:56666,mumbai:26989","1:3257,3:9162,2:12893,5:9237,4:8503,7:25094,6:...","Drama:48949,Family:9927,Crime:1901,Romance:165...",0,"Naamkarann:369,Ishq:59,Pardes Mein Hai Mera Di...","11:7726,10:3521,13:2067,12:3844,20:4608,21:936...",121673
2,navi mumbai:29400,"1:40,3:2700,2:4659,5:7338,4:6641,7:2218,6:5800","Cricket:5715,Family:2777,Drama:20873,LiveTV:4,...",0,"India vs Australia 1st Test Hindi:83,India vs ...","11:302,10:1594,13:331,12:133,20:3802,14:3083,2...",121670
3,"chennai:77,navi mumbai:4005,mumbai:0","1:77,2:3883,5:122","Romance:77,Drama:122,Cricket:3883",0,"Rangoon:77,India A vs England XI:3883,Kaabil:122","10:122,12:77,15:1800,14:0,16:2083",121671
4,"bangalore:5135,krishnarajapura:529,hosur:2864","1:6150,3:2378",TalkShow:8529,1,Koffee With Karan:8529,"20:529,21:1257,22:4930,23:538,19:1268,18:6",121676


In [83]:
train = pd.concat([df_500,city,genre,days,time_of_day],axis = 1)
drop = ['cities', 'dow', 'genres', 'titles', 'tod']
train = train.drop(drop,1)
train = train.apply(pd.to_numeric)

In [84]:
train.head(10) #Great now that's our final df and we will be using all columns or features

,segment,user,delhi,mumbai,ahmedabad,navi mumbai,chennai,bangalore,krishnarajapura,hosur,...,2 am,9 am,3 pm,2 pm,8 am,7 am,6 am,5 am,3 am,4 am
0,0,121672,3492,64,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,121673,0,26989,56666,0,0,0,0,0,...,9807,2005,967,2807,0,0,0,0,0,0
2,0,121670,0,0,0,29400,0,0,0,0,...,0,4388,1640,3083,201,504,1874,0,0,0
3,0,121671,0,0,0,4005,77,0,0,0,...,0,0,1800,0,0,0,0,0,0,0
4,1,121676,0,0,0,0,0,5135,529,2864,...,0,0,0,0,0,0,0,0,0,0
5,0,121677,0,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,121674,13142,0,0,0,0,0,0,0,...,0,0,2158,4446,0,0,1648,1490,0,0
7,0,121675,0,77,0,0,6205,373,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,121678,9492,19504,0,0,0,0,0,0,...,4559,0,0,683,10,0,0,0,2556,0
9,0,121679,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
train["segment"].value_counts() 

0    442
1     58
Name: segment, dtype: int64

In [86]:
#Unfortunately, we will have to do the same thing for the test set as well. Let's quickly get to that 
#and name the final_df as test

In [89]:
df = pd.read_json("/Users/amitagarwal/GreyAtom/Hackathon/Hotstar/data/test_data.json")
df = df.T


In [90]:
df.shape

(100000, 5)

In [91]:
test_copy = df.copy()

In [92]:
df = df.iloc[:500] #Take the first 500 rows of the test set

In [93]:
cities = df["cities"].values.tolist()


In [94]:
cities

['delhi:3890,chennai:12,navi mumbai:22650',
 'navi mumbai:4052',
 'hyderabad:2922,delhi:504,mumbai:1060',
 'thiruvananthapuram:1021',
 'navi mumbai:9381',
 'Unknown:16986,chennai:20',
 'karachi:8082',
 'dubayy:182',
 'delhi:1186',
 'navi mumbai:20481',
 'mumbai:98',
 'hyderabad:806,navi mumbai:2',
 'new delhi:1262,bangalore:2962,bhopal:22818',
 'new delhi:181,gurgaon:953,delhi:37',
 'dubayy:64797,bangalore:215',
 'dhaka:1805',
 'ulhasnagar:4,mumbai:2058',
 'chandigarh:4613,delhi:10821,navi mumbai:68,mumbai:194',
 'navi mumbai:1006',
 'hyderabad:54690,bangalore:1372,delhi:3194,navi mumbai:22067,los angeles:633',
 'navi mumbai:6',
 'delhi:27335',
 'navi mumbai:39684',
 'navi mumbai:2315',
 'navi mumbai:30619',
 'delhi:9116,mumbai:13714',
 'new delhi:4766,delhi:5698,mumbai:7999',
 'new delhi:11108,bangalore:15669,delhi:6699,chennai:3612,mumbai:7743',
 'Unknown:3297,delhi:28',
 'Unknown:2511,secunderabad:1858',
 'delhi:7',
 'dhaka:1076',
 'bhopal:10',
 'navi mumbai:1090',
 'bangalore:4833,

In [95]:

final=[]
for i in range(len(cities)):
    frames=[]
    user_cities = cities[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_cities] for item in sublist]
    city = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([city,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True, sort=False)
    final.append(pd.DataFrame(final_df))
    
city = pd.concat(final,join='outer', ignore_index=True, sort=False)
city = city.fillna(0) 


In [96]:
genres = df["genres"].values.tolist()
final_1=[]
for i in range(len(genres)):
    frames=[]
    genre = genres[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in genre] for item in sublist]
    flat_list = [i.lower() for i in flat_list]
    genre_1 = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    genre_df = pd.concat([genre_1,second],ignore_index = True)
    genre_df.columns = genre_df.iloc[0]
    genre_df = genre_df.drop([0],0)
    frames.append(genre_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True, sort=False)
    final_1.append(pd.DataFrame(final_df))
    
genre = pd.concat(final_1,join='outer', ignore_index=True, sort=False)
genre = genre.fillna(0) 

days_of_week = df["dow"].values.tolist()
    
final_2=[]
for i in range(len(days_of_week)):
    frames=[]
    user_days = days_of_week[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_days] for item in sublist]
#    flat_list = [int(i) for i in flat_list]
    day = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([day,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True, sort=False)
    final_2.append(pd.DataFrame(final_df))

days = pd.concat(final_2,join='outer', ignore_index=True, sort=False)
days = days.fillna(0)
    
days_dict = {"1":"Sunday", "2":"Monday", "3":"Tuesday", "4":"Wednesday", "5":"Thursday", 
             "6":"Friday", "7":"Saturday"} 
    
day_key = pd.DataFrame(list(days_dict.items()),columns = ["dow_cat","day"])  
d = day_key.set_index('dow_cat').to_dict()  
days.columns = days.columns.to_series().map(d['day'])
#time_dict = {"0":"12 am",
#             "1":"1 am"
#             "2":"2 am"
#             "3":"3 am"
#             "4":"4 am"
#        }

time_am = [str(x) for x in range(0,12)]
time_pm=[str(x) for x in range(12,24)]
am=[str(x)+ " am" for x in range(0,12)]
pm=[str(x)+ " pm" for x in range(0,12)]
d1 = dict(zip(time_am,am))
d2 = dict(zip(time_pm,pm))
d1["0"] = "12 am"
d2["12"] = "12 pm"
time_dict = {**d1,**d2}

time_of_day = df["tod"].values.tolist()

final_3=[]
for i in range(len(days_of_week)):
    frames=[]
    user_time = time_of_day[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_time] for item in sublist]
#    flat_list = [int(i) for i in flat_list]
    time = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([time,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True, sort=False)
    final_3.append(pd.DataFrame(final_df))
timeofday = pd.concat(final_3,join='outer', ignore_index=True, sort=False)
timeofday = timeofday.fillna(0)
time_key = pd.DataFrame(list(time_dict.items()),columns = ["tod_cat","hour"]) 
t = time_key.set_index('tod_cat').to_dict() 
timeofday.columns = timeofday.columns.to_series().map(t['hour']) 
time_of_day = timeofday



In [97]:



df["index"] = df.index
df["user"] = df["index"].str.replace("train-","")

df = df.reset_index()




In [98]:
test = pd.concat([df,city,genre,days,time_of_day],axis = 1)
drop = ['level_0', 'cities', 'dow', 'genres', 'titles', 'tod', 'index']
test = test.drop(drop,1)
test["user"] = test["user"].str.replace("test-","")

In [99]:
test = test.apply(pd.to_numeric)

In [100]:
test.head()

,user,delhi,chennai,navi mumbai,hyderabad,mumbai,thiruvananthapuram,Unknown,karachi,dubayy,...,2 am,7 am,6 am,9 am,8 am,2 pm,3 pm,3 am,5 am,4 am
0,23855,3890,12,22650,0,0,0,0,0,0,...,567,340,1330,304,684,817,0,0,0,0
1,23854,0,0,4052,0,0,0,0,0,0,...,0,0,0,0,0,0,775,0,0,0
2,23857,504,0,0,2922,1060,0,0,0,0,...,0,0,0,0,0,0,37,0,0,0
3,23856,0,0,0,0,0,1021,0,0,0,...,0,0,0,0,0,0,91,0,0,0
4,23851,0,0,9381,0,0,0,0,0,0,...,0,0,0,0,0,0,244,0,0,0


In [101]:
#Now our train consists of 2000 data points and test of 500

In [102]:
#Let's do a small train accuracy finding exercise first. We will see the first exerecise on a training set of 500
#samples only and run a log reg. Basically we have 500 samples from the train, we will break this down into test and train
#and run a 
#log reg

In [103]:
train.head(5)

,segment,user,delhi,mumbai,ahmedabad,navi mumbai,chennai,bangalore,krishnarajapura,hosur,...,2 am,9 am,3 pm,2 pm,8 am,7 am,6 am,5 am,3 am,4 am
0,0,121672,3492,64,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,121673,0,26989,56666,0,0,0,0,0,...,9807,2005,967,2807,0,0,0,0,0,0
2,0,121670,0,0,0,29400,0,0,0,0,...,0,4388,1640,3083,201,504,1874,0,0,0
3,0,121671,0,0,0,4005,77,0,0,0,...,0,0,1800,0,0,0,0,0,0,0
4,1,121676,0,0,0,0,0,5135,529,2864,...,0,0,0,0,0,0,0,0,0,0


In [104]:
from sklearn.model_selection import train_test_split as tts
y = train["segment"]
X = train.drop(["segment"],1)
print (X.shape)
print (df.shape)
y.value_counts()

(500, 118)
(500, 8)


0    442
1     58
Name: segment, dtype: int64

In [105]:
from imblearn.over_sampling import SMOTE

In [106]:
sm = SMOTE(random_state=42)

In [107]:
X_train,X_test,y_train,y_test = tts(X,y,test_size = 0.2,random_state = 42)

In [108]:
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

In [109]:
cols = X_train.columns

In [110]:
X_train_res = pd.DataFrame(X_train_res,columns = cols)

In [111]:
y_train_res = pd.Series(y_train_res)

In [112]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()

In [113]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [114]:
log_reg.fit(X_train_res,y_train_res)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [115]:
y_pred = log_reg.predict(X_test)

In [116]:
print (accuracy_score(y_test,y_pred)) #Gives us 0.86 for 500 train samples with log reg

0.84


In [117]:
#Good, Let's increase the train size to 2000 samples and see the training accuracy with a log reg

#Decreases to 65%. Putting a decision tree instead

In [118]:
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier(min_samples_leaf=0.20, max_depth=40, min_samples_split=4)
dc.fit(X_train_res,y_train_res)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=40,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=0.2, min_samples_split=4,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [119]:
y_pred = dc.predict(X_test)

In [120]:
accuracy_score(y_test,y_pred)

0.56

In [121]:
from sklearn.ensemble import RandomForestClassifier


In [122]:
rfc= RandomForestClassifier(n_estimators=100)

In [123]:
rfc.fit(X_train_res,y_train_res)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [124]:
y_pred = rfc.predict(X_test)

In [125]:
accuracy_score(y_test,y_pred) #Great. Gives us a 87% 

0.88